In [1]:
import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.nn import Linear

from torch_geometric.data import TemporalData
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [2]:
data = torch.load("data/act-mooc/graph.pt")
#data = torch.load("data/junyi/graph.pt")

del data[('resource', 'rev_accesses', 'user')]
data_hom = data.to_homogeneous()
data_temp = TemporalData(
    src=data_hom.edge_index[0,:].to(torch.long),
    dst=data_hom.edge_index[1,:].to(torch.long),
    t=data_hom.time.to(torch.long),
    msg=data_hom.edge_attr.to(torch.float),
    y=data_hom.edge_y.to(torch.long)
)
data = data_temp

define data loaders

In [3]:
# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
data = data.to(device)

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15)

batch_size = 20

train_loader = TemporalDataLoader(
    train_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)
neighbor_loader = LastNeighborLoader(data.num_nodes, size=10, device=device)

In [4]:
class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super().__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels // 2, heads=2,
                                    dropout=0.1, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        return self.conv(x, edge_index, edge_attr)


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.lin_src = Linear(in_channels, in_channels)
        self.lin_dst = Linear(in_channels, in_channels)
        self.lin_final = Linear(in_channels, 1)

    def forward(self, z_src, z_dst):
        h = self.lin_src(z_src) + self.lin_dst(z_dst)
        h = h.relu()
        return self.lin_final(h)

In [5]:
memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    data.num_nodes,
    data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)
link_pred_exist = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)


def train():
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.

    batches = 0
    total_loss = 0
    for batch in train_loader:
        batches += 1
        print(batches)
        if batches == 10:
            break
        optimizer.zero_grad()
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        print(batch.edge_index)
        print("edge_index")
        print(edge_index)

        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))

        positive_edges = batch.edge_index[:, batch.y == 1]
        pos_out = link_pred(z[assoc[positive_edges[0]]], z[assoc[positive_edges[1]]])
        loss = criterion(pos_out, torch.ones_like(pos_out))
        print(positive_edges.shape)
        
        negative_indices = torch.nonzero(batch.y == 0).squeeze()
        negative_indices = negative_indices[torch.randperm(negative_indices.size(0))][:positive_edges.size(1)]
        negative_edges = batch.edge_index[:, negative_indices]
        print(negative_edges.shape)
        neg_out = link_pred(z[assoc[negative_edges[0]]], z[assoc[negative_edges[1]]])
        loss += criterion(neg_out, torch.zeros_like(neg_out))

        # maybe it helps to still predict the existence of edges with another link predictor
        #pos_out = link_pred_exist(z[assoc[batch.src]], z[assoc[batch.dst]])
        #neg_out = link_pred_exist(z[assoc[batch.src]], z[assoc[batch.neg_dst]])
        #loss = criterion(pos_out, torch.ones_like(pos_out))
        #loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)

        loss.backward()
        optimizer.step()
        memory.detach()
        total_loss += float(loss) * batch.num_events

    return total_loss / train_data.num_events


@torch.no_grad()
def test(loader):
    memory.eval()
    gnn.eval()
    link_pred.eval()

    torch.manual_seed(12345)  # Ensure deterministic sampling across epochs.

    aps, aucs = [], []
    for batch in loader:
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        
        out = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]])
        #pos_out = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]])
        #neg_out = link_pred(z[assoc[batch.src]], z[assoc[batch.neg_dst]])

        y_pred = out.sigmoid().cpu()
        y_true = batch.y
        #y_pred = torch.cat([pos_out, neg_out], dim=0).sigmoid().cpu()
        #y_true = torch.cat(
        #    [torch.ones(pos_out.size(0)),
        #     torch.zeros(neg_out.size(0))], dim=0)

        aps.append(average_precision_score(y_true, y_pred))
        aucs.append(roc_auc_score(y_true, y_pred))

        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)
    return float(torch.tensor(aps).mean()), float(torch.tensor(aucs).mean())



In [6]:
for epoch in range(1, 2):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
    val_ap, val_auc = test(val_loader)
    test_ap, test_auc = test(test_loader)
    print(f'Val AP: {val_ap:.4f}, Val AUC: {val_auc:.4f}')
    print(f'Test AP: {test_ap:.4f}, Test AUC: {test_auc:.4f}')

1
tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,    1,    1],
        [7047, 7048, 7049, 7048, 7049, 7050, 7051, 7052, 7053, 7054, 7055, 7056,
         7047, 7056, 7049, 7048, 7057, 7057, 7048, 7049]])
edge_index
tensor([], size=(2, 0), dtype=torch.int64)
torch.Size([2, 0])
torch.Size([2, 0])
2
tensor([[   1,    1,    1,    1,    1,    2,    3,    4,    4,    3,    3,    5,
            5,    6,    7,    5,    3,    8,    6,    7],
        [7054, 7047, 7058, 7059, 7058, 7048, 7048, 7048, 7050, 7057, 7050, 7048,
         7050, 7048, 7048, 7055, 7060, 7048, 7050, 7048]])
edge_index
tensor([[11, 10, 15,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0],
        [ 1,  1,  1,  9,  9, 10, 10, 10, 10, 12, 13, 14, 15, 15]])
torch.Size([2, 0])
torch.Size([2, 0])
3
tensor([[   6,    6,    6,    9,    7,    7,    6,    7,    6,    6,    6,    6,
           10,    6,   10,    6,    4,    4,    4,    4],
        [7048, 7049, 70